In [1]:
import pandas as pd
from functools import reduce
import numpy as np

In [2]:
Movement = pd.read_csv(r"../FilesCSVFormat/ArtistMovements.csv")
Artwork = pd.read_csv("../FilesCSVFormat/Artwork.csv")
Special = pd.read_csv("../FilesCSVFormat/ArtistSpecializations.csv")
Generated = pd.read_csv("../FilesCSVFormat/Generated.csv")
Artist = pd.read_csv("../FilesCSVFormat/Artist.csv")
Education = pd.read_csv("../FilesCSVFormat/ArtistEducation.csv")
Places = pd.read_csv("../FilesCSVFormat/Places.csv")
OutputReal = pd.read_csv("../GAN_Detection/output_real.csv")
OutputFake = pd.read_csv("../GAN_Detection/output_fake.csv")
FeaturesFake = pd.read_csv("../Image Features/output/image_features_fake.csv")
FeaturesReal = pd.read_csv("../Image Features/output/image_features_real.csv")
Rec = pd.read_csv("../n_relations_recommendations.csv")

In [3]:
Education.head(10)

,artist_id,academy_id
0,0.0,0.0
1,0.0,1.0
2,2.0,2.0
3,2.0,3.0
4,9.0,4.0
5,13.0,5.0
6,19.0,6.0
7,19.0,2.0
8,26.0,7.0
9,27.0,8.0


# Artwork

In [4]:
Artwork.shape

(10517, 10)

In [5]:
pd.set_option('max_colwidth', 50)
Artwork.head(3)

,id,name,url,image_url,artist,rating,summary,year,medium,location
0,0,Cafe Terrace on the Place du Forum,http://wikigallery.org/wiki/painting_133032/Vi...,https://kuleuven-datathon-2023.s3.eu-central-1...,0,8.0,Café Terrace at Night is an 1888 oil painting ...,1888.0,0.0,Kröller-Müller Museum
1,1,Starry Night,http://wikigallery.org/wiki/painting_9698/Vinc...,https://kuleuven-datathon-2023.s3.eu-central-1...,0,9.0,NaN,NaN,NaN,NaN
2,2,A Digger,http://wikigallery.org/wiki/painting_132953/Vi...,https://kuleuven-datathon-2023.s3.eu-central-1...,0,7.0,NaN,NaN,NaN,NaN


In [6]:
Artwork["year"].isna().sum()/len(Artwork) #should be removed due to large number of NAs 

0.9751830369877341

In [7]:
Artwork["medium"].isna().sum()/len(Artwork) #should be removed due to large number of NAs 

0.9499857373775792

In [8]:
Artwork["location"].isna().sum()/len(Artwork) #should be removed due to large number of NAs 

0.9395264809356281

In [9]:
Artwork2 = Artwork.iloc[:, [0,1,3,4,5]]
Artwork2 = Artwork2.rename(columns = {'artist':'artist_id'})
Artwork2.sample(5)

,id,name,image_url,artist_id,rating
9109,9110,The Family Gathering,https://kuleuven-datathon-2023.s3.eu-central-1...,520,NaN
9850,9851,Still life with wine and grapes,https://kuleuven-datathon-2023.s3.eu-central-1...,568,NaN
5416,5416,Resting in the lane,https://kuleuven-datathon-2023.s3.eu-central-1...,299,10.0
10432,10435,"Portrait of Mouton playing the lute, engraved ...",https://kuleuven-datathon-2023.s3.eu-central-1...,605,NaN
10491,10494,Les quatre Ages de la vie Seven figures in a g...,https://kuleuven-datathon-2023.s3.eu-central-1...,591,NaN


In [10]:
Artwork2["AI_Generated"] = 0
Artwork2.head()

,id,name,image_url,artist_id,rating,AI_Generated
0,0,Cafe Terrace on the Place du Forum,https://kuleuven-datathon-2023.s3.eu-central-1...,0,8.0,0
1,1,Starry Night,https://kuleuven-datathon-2023.s3.eu-central-1...,0,9.0,0
2,2,A Digger,https://kuleuven-datathon-2023.s3.eu-central-1...,0,7.0,0
3,3,A Group of Cottages,https://kuleuven-datathon-2023.s3.eu-central-1...,0,6.0,0
4,4,A Pair of Shoes,https://kuleuven-datathon-2023.s3.eu-central-1...,0,8.0,0


In [11]:
pd.set_option('max_colwidth', 200)
Artwork["image_url"].head(5)

0    https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/Cafe+Terrace+on+the+Place+du+Forum.jpg
1                          https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/Starry+Night.jpg
2                              https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/A+Digger.jpg
3                   https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/A+Group+of+Cottages.jpg
4                       https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/A+Pair+of+Shoes.jpg
Name: image_url, dtype: object

# Generated

In [12]:
Generated.shape

(3447, 2)

In [13]:
Generated.head()

,source_artwork,url
0,3148,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/generated/A painting of 6th Sokol Festival. 1912 in the style of Alphonse Maria Mucha.png
1,1480,"https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/generated/A painting of A Back Road, c.1880s in the style of Childe Hassam.png"
2,182,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/generated/A painting of A Backwater at Calcot Near Reading in the style of John Singer Sargent.png
3,2493,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/generated/A painting of A Ball During the Reign of Henri IV in the style of Louis de Caullery.png
4,2497,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/generated/A painting of A Banquet Scene in the style of Louis de Caullery.png


In [14]:
Generated["AI_Generated"] = 1 
Generated2 = Generated.rename(columns = {'url':'image_url'})
Generated2.head()

,source_artwork,image_url,AI_Generated
0,3148,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/generated/A painting of 6th Sokol Festival. 1912 in the style of Alphonse Maria Mucha.png,1
1,1480,"https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/generated/A painting of A Back Road, c.1880s in the style of Childe Hassam.png",1
2,182,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/generated/A painting of A Backwater at Calcot Near Reading in the style of John Singer Sargent.png,1
3,2493,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/generated/A painting of A Ball During the Reign of Henri IV in the style of Louis de Caullery.png,1
4,2497,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/generated/A painting of A Banquet Scene in the style of Louis de Caullery.png,1


# Movement

In [15]:
Movement.head(15)

,artist_id,movement_id
0,0.0,43.0
1,1.0,44.0
2,2.0,44.0
3,4.0,10.0
4,4.0,52.0
5,6.0,32.0
6,6.0,25.0
7,8.0,4.0
8,8.0,14.0
9,8.0,37.0


In [16]:
Movement.shape

(255, 2)

# Specialization 

In [17]:
Special.head()

,artist_id,specialty_id
0,0.0,0.0
1,0.0,1.0
2,0.0,2.0
3,0.0,3.0
4,1.0,2.0


# Artist

In [18]:
Artist.sample(5)

,id,name,url,summary,picture,birthplace,deathplace,birthdate,deathdate,cause_of_death
230,230,Paul Klee,https://www.wikigallery.org//wiki/artist36780/Paul-Klee/page-1,"Paul Klee (German: [paʊ̯l ˈkleː]; 18 December 1879 – 29 June 1940) was a Swiss-born German artist. His highly individual style was influenced by movements in art that included expressionism, cubis...",230.0,105.0,397.0,NaN,1940-06-29,NaN
222,222,Franz Xaver Messerschmidt,https://www.wikigallery.org//wiki/artist46925/Franz-Xaver-Messerschmidt/page-1,"Franz Xaver Messerschmidt (February 6, 1736 – August 19, 1783) was a German-Austrian sculptor most famous for his ""character heads"", a collection of busts with faces contorted in extreme facial ex...",222.0,101.0,395.0,NaN,1783-08-19,NaN
92,92,Raja Ravi Varma,https://www.wikigallery.org//wiki/artist43738/Raja-Ravi-Varma/page-1,Raja Ravi Varma (29 April 1848 – 2 October 1906 ) was an Indian painter and artist. He is considered among the greatest painters in the history of Indian art. His works are one of the best example...,92.0,57.0,372.0,1848-04-29,1906-10-02,NaN
491,491,William Stephen Coleman,https://www.wikigallery.org//wiki/artist46212/William-Stephen-Coleman/page-1,William Stephen Coleman (1829–1904) was an English painter and book illustrator.,NaN,NaN,NaN,NaN,NaN,NaN
150,150,Angelica Kauffmann,https://www.wikigallery.org//wiki/artist34533/Angelica-Kauffmann/page-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
Artist["cause_of_death"].isna().sum()/len(Artist) #cause of data column should be removed due to large number of NAs

0.9967532467532467

In [20]:
Artist["birthplace"].isna().sum()/len(Artist)

0.5

In [21]:
Artist["deathplace"].isna().sum()/len(Artist)

0.5275974025974026

In [22]:
Artist["birthdate"].isna().sum()/len(Artist)

0.6347402597402597

In [23]:
Artist["deathdate"].isna().sum()/len(Artist)

0.5616883116883117

In [24]:
Artist2 = Artist.iloc[:,[0,4,5,6,7,8]]
Artist2 = Artist2.rename(columns = {"id": "artist_id"})
Artist2['artist_id'] = Artist2['artist_id'].astype(float)
Artist2.head()

,artist_id,picture,birthplace,deathplace,birthdate,deathdate
0,0.0,0.0,0.0,342.0,1853-03-30,1890-07-29
1,1.0,1.0,1.0,343.0,1841-02-25,1919-12-03
2,2.0,2.0,2.0,344.0,1840-11-14,1926-12-05
3,3.0,3.0,3.0,NaN,1932-04-19,NaN
4,4.0,4.0,4.0,2.0,1824-05-11,1904-01-10


# Artist Education

In [25]:
Education.head()

,artist_id,academy_id
0,0.0,0.0
1,0.0,1.0
2,2.0,2.0
3,2.0,3.0
4,9.0,4.0


In [26]:
Education = Education.astype(int)
Education2 = Education.pivot_table(index = "artist_id", columns = "academy_id", aggfunc = "size", fill_value = 0)
Education2

academy_id,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
artist_id,,,,,,,,,,,,,,,,,,,,,
0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
557,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
572,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
Education2 = Education2.add_prefix("Education_").reset_index(level=0)
Education2 = Education2.reset_index(drop=True)
Education2.columns.name = None
Education2

,artist_id,Education_0,Education_1,Education_2,Education_3,Education_4,Education_5,Education_6,Education_7,Education_8,...,Education_38,Education_39,Education_40,Education_41,Education_42,Education_43,Education_44,Education_45,Education_46,Education_47
0,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,9,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,13,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,19,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,545,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
58,557,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
59,572,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60,604,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Places

In [28]:
Places.head()

,id,name,parent
0,0,Zundert,213.0
1,1,Limoges,214.0
2,2,Paris,214.0
3,3,Medellín,215.0
4,4,Vesoul,216.0


In [29]:
Places2 = Places.iloc[:,[0,2]]
Places2 = Places2.rename(columns = {"id": "artist_id"})

# Recommendation

In [30]:
Rec.head()

,id,num_rels
0,0,1
1,1,1
2,2,5
3,3,0
4,4,0


# Text split

In [31]:
def FindNumber(inputstr):
    return inputstr.split("\\")[-1].split(".")[0]

In [32]:
OutputReal["filename"] = OutputReal.apply(lambda row: FindNumber(row['filename']), axis = 1)

In [33]:
OutputReal2 = OutputReal.rename(columns = {"filename":"id", "logit": "real_logit", "time":"real_time"})

In [34]:
OutputReal2['id'] = OutputReal2['id'].astype(float) #change to float values to match main dataframe

In [35]:
OutputFake["filename"] = OutputFake.apply(lambda row: FindNumber(row['filename']), axis = 1)

In [36]:
OutputFake2 = OutputFake.rename(columns = {"filename":"id", "logit": "fake_logit", "time":"fake_time"})

In [37]:
OutputFake2['id'] = OutputFake2['id'].astype(float) #change to float values to match main dataframe

# Merge Dataframes

In [38]:
Artwork2.head()

,id,name,image_url,artist_id,rating,AI_Generated
0,0,Cafe Terrace on the Place du Forum,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/Cafe+Terrace+on+the+Place+du+Forum.jpg,0,8.0,0
1,1,Starry Night,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/Starry+Night.jpg,0,9.0,0
2,2,A Digger,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/A+Digger.jpg,0,7.0,0
3,3,A Group of Cottages,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/A+Group+of+Cottages.jpg,0,6.0,0
4,4,A Pair of Shoes,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/A+Pair+of+Shoes.jpg,0,8.0,0


In [39]:
OutputReal2['id'] = OutputReal2['id'].astype(int)
OutputReal2.head()

,id,real_logit,real_time
0,0,-8.378802,6.175854
1,1,-14.644643,6.019991
2,10,-6.296502,6.170134
3,100,-8.054534,6.067291
4,1000,-1.293363,5.589392


In [40]:
Artwork3 = pd.merge(Artwork2, OutputReal2, on='id')


In [41]:
Artwork3.head()

,id,name,image_url,artist_id,rating,AI_Generated,real_logit,real_time
0,0,Cafe Terrace on the Place du Forum,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/Cafe+Terrace+on+the+Place+du+Forum.jpg,0,8.0,0,-8.378802,6.175854
1,1,Starry Night,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/Starry+Night.jpg,0,9.0,0,-14.644643,6.019991
2,2,A Digger,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/A+Digger.jpg,0,7.0,0,-4.634872,5.785132
3,3,A Group of Cottages,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/A+Group+of+Cottages.jpg,0,6.0,0,-10.061778,6.172345
4,4,A Pair of Shoes,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/A+Pair+of+Shoes.jpg,0,8.0,0,-5.156153,6.156727


In [42]:
Artwork3.head()

,id,name,image_url,artist_id,rating,AI_Generated,real_logit,real_time
0,0,Cafe Terrace on the Place du Forum,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/Cafe+Terrace+on+the+Place+du+Forum.jpg,0,8.0,0,-8.378802,6.175854
1,1,Starry Night,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/Starry+Night.jpg,0,9.0,0,-14.644643,6.019991
2,2,A Digger,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/A+Digger.jpg,0,7.0,0,-4.634872,5.785132
3,3,A Group of Cottages,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/A+Group+of+Cottages.jpg,0,6.0,0,-10.061778,6.172345
4,4,A Pair of Shoes,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/A+Pair+of+Shoes.jpg,0,8.0,0,-5.156153,6.156727


In [43]:
Artwork4 = pd.merge(Artwork3, FeaturesReal, on='id')
Artwork4 = Artwork4.rename(columns={'real_logit': 'logit', 'real_time': 'time'})
Artwork4.head()

,id,name,image_url,artist_id,rating,AI_Generated,logit,time,edge_score,contrast_score,brightness_score,saturation_score
0,0,Cafe Terrace on the Place du Forum,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/Cafe+Terrace+on+the+Place+du+Forum.jpg,0,8.0,0,-8.378802,6.175854,46.383220,55.552072,112.148305,127.663731
1,1,Starry Night,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/Starry+Night.jpg,0,9.0,0,-14.644643,6.019991,47.354136,54.624181,124.601402,62.417748
2,2,A Digger,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/A+Digger.jpg,0,7.0,0,-4.634872,5.785132,28.550276,48.946494,103.395141,80.945548
3,3,A Group of Cottages,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/A+Group+of+Cottages.jpg,0,6.0,0,-10.061778,6.172345,67.087002,52.743048,166.384401,80.268604
4,4,A Pair of Shoes,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/A+Pair+of+Shoes.jpg,0,8.0,0,-5.156153,6.156727,58.546688,54.431662,87.995360,159.627900


In [44]:
Artwork5 = pd.merge(Artwork4, Rec, on='id', how = 'left')
Artwork5.head()

,id,name,image_url,artist_id,rating,AI_Generated,logit,time,edge_score,contrast_score,brightness_score,saturation_score,num_rels
0,0,Cafe Terrace on the Place du Forum,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/Cafe+Terrace+on+the+Place+du+Forum.jpg,0,8.0,0,-8.378802,6.175854,46.383220,55.552072,112.148305,127.663731,1
1,1,Starry Night,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/Starry+Night.jpg,0,9.0,0,-14.644643,6.019991,47.354136,54.624181,124.601402,62.417748,1
2,2,A Digger,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/A+Digger.jpg,0,7.0,0,-4.634872,5.785132,28.550276,48.946494,103.395141,80.945548,5
3,3,A Group of Cottages,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/A+Group+of+Cottages.jpg,0,6.0,0,-10.061778,6.172345,67.087002,52.743048,166.384401,80.268604,0
4,4,A Pair of Shoes,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/A+Pair+of+Shoes.jpg,0,8.0,0,-5.156153,6.156727,58.546688,54.431662,87.995360,159.627900,0


In [45]:
Generated2.head()

,source_artwork,image_url,AI_Generated
0,3148,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/generated/A painting of 6th Sokol Festival. 1912 in the style of Alphonse Maria Mucha.png,1
1,1480,"https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/generated/A painting of A Back Road, c.1880s in the style of Childe Hassam.png",1
2,182,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/generated/A painting of A Backwater at Calcot Near Reading in the style of John Singer Sargent.png,1
3,2493,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/generated/A painting of A Ball During the Reign of Henri IV in the style of Louis de Caullery.png,1
4,2497,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/generated/A painting of A Banquet Scene in the style of Louis de Caullery.png,1


In [46]:
Generated2 = Generated2.rename(columns = {'source_artwork':'id'})
Generated2.head()

,id,image_url,AI_Generated
0,3148,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/generated/A painting of 6th Sokol Festival. 1912 in the style of Alphonse Maria Mucha.png,1
1,1480,"https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/generated/A painting of A Back Road, c.1880s in the style of Childe Hassam.png",1
2,182,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/generated/A painting of A Backwater at Calcot Near Reading in the style of John Singer Sargent.png,1
3,2493,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/generated/A painting of A Ball During the Reign of Henri IV in the style of Louis de Caullery.png,1
4,2497,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/generated/A painting of A Banquet Scene in the style of Louis de Caullery.png,1


In [47]:
Generated3 = pd.merge(Generated2, Artwork5[['id','artist_id','num_rels']], on='id', how='left')
Generated3.head()

,id,image_url,AI_Generated,artist_id,num_rels
0,3148,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/generated/A painting of 6th Sokol Festival. 1912 in the style of Alphonse Maria Mucha.png,1,146,1
1,1480,"https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/generated/A painting of A Back Road, c.1880s in the style of Childe Hassam.png",1,59,0
2,182,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/generated/A painting of A Backwater at Calcot Near Reading in the style of John Singer Sargent.png,1,7,0
3,2493,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/generated/A painting of A Ball During the Reign of Henri IV in the style of Louis de Caullery.png,1,106,2
4,2497,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/generated/A painting of A Banquet Scene in the style of Louis de Caullery.png,1,106,0


In [48]:
# merge the dataframes
Generated4 = pd.merge(Generated3, OutputFake2, on='id', how='left')
Generated4.head()


,id,image_url,AI_Generated,artist_id,num_rels,fake_logit,fake_time
0,3148,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/generated/A painting of 6th Sokol Festival. 1912 in the style of Alphonse Maria Mucha.png,1,146,1,0.662618,5.648655
1,1480,"https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/generated/A painting of A Back Road, c.1880s in the style of Childe Hassam.png",1,59,0,-4.706479,4.893455
2,182,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/generated/A painting of A Backwater at Calcot Near Reading in the style of John Singer Sargent.png,1,7,0,-2.122950,4.907467
3,2493,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/generated/A painting of A Ball During the Reign of Henri IV in the style of Louis de Caullery.png,1,106,2,0.092303,4.893453
4,2497,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/generated/A painting of A Banquet Scene in the style of Louis de Caullery.png,1,106,0,0.323381,4.949506


In [49]:
Generated4.shape

(3447, 7)

In [50]:
Generated5 = pd.merge(Generated4, FeaturesFake, on='id', how='left')

Generated5 = Generated5.rename(columns={'fake_logit': 'logit', 'fake_time': 'time'})
Generated5.head()

,id,image_url,AI_Generated,artist_id,num_rels,logit,time,edge_score,contrast_score,brightness_score,saturation_score
0,3148,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/generated/A painting of 6th Sokol Festival. 1912 in the style of Alphonse Maria Mucha.png,1,146,1,0.662618,5.648655,51.999142,60.835517,128.136902,99.981544
1,1480,"https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/generated/A painting of A Back Road, c.1880s in the style of Childe Hassam.png",1,59,0,-4.706479,4.893455,52.029572,74.418921,104.963806,117.600163
2,182,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/generated/A painting of A Backwater at Calcot Near Reading in the style of John Singer Sargent.png,1,7,0,-2.122950,4.907467,21.968922,55.114939,122.415871,114.433147
3,2493,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/generated/A painting of A Ball During the Reign of Henri IV in the style of Louis de Caullery.png,1,106,2,0.092303,4.893453,26.734337,82.190911,125.295952,77.526588
4,2497,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/generated/A painting of A Banquet Scene in the style of Louis de Caullery.png,1,106,0,0.323381,4.949506,33.706455,57.276167,98.605370,175.421700


In [51]:
Generated5.shape

(3447, 11)

In [52]:
df_list1 = [Artwork5, Generated5]
df1 = pd.concat(df_list1)
df1.head()
df1.to_csv('df1.csv')

In [53]:
df1.head()

,id,name,image_url,artist_id,rating,AI_Generated,logit,time,edge_score,contrast_score,brightness_score,saturation_score,num_rels
0,0,Cafe Terrace on the Place du Forum,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/Cafe+Terrace+on+the+Place+du+Forum.jpg,0,8.0,0,-8.378802,6.175854,46.383220,55.552072,112.148305,127.663731,1
1,1,Starry Night,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/Starry+Night.jpg,0,9.0,0,-14.644643,6.019991,47.354136,54.624181,124.601402,62.417748,1
2,2,A Digger,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/A+Digger.jpg,0,7.0,0,-4.634872,5.785132,28.550276,48.946494,103.395141,80.945548,5
3,3,A Group of Cottages,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/A+Group+of+Cottages.jpg,0,6.0,0,-10.061778,6.172345,67.087002,52.743048,166.384401,80.268604,0
4,4,A Pair of Shoes,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/A+Pair+of+Shoes.jpg,0,8.0,0,-5.156153,6.156727,58.546688,54.431662,87.995360,159.627900,0


In [54]:
df1.shape

(13961, 13)

In [55]:
Artwork.shape + Generated.shape
10517+3447

13964

In [56]:
Movement = Movement.astype(int)

In [57]:
Movement2 = Movement.pivot_table(index = "artist_id", columns = "movement_id", aggfunc = "size", fill_value = 0)
Movement2 = Movement2.add_prefix("Movement_").reset_index(level=0)
Movement2 = Movement2.reset_index(drop=True)
Movement2.columns.name = None
Movement2

,artist_id,Movement_0,Movement_1,Movement_2,Movement_3,Movement_4,Movement_5,Movement_6,Movement_7,Movement_8,...,Movement_45,Movement_46,Movement_47,Movement_48,Movement_49,Movement_50,Movement_51,Movement_52,Movement_53,Movement_54
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,604,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
187,606,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
188,609,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
189,610,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
Special = Special.astype(int)
Special.head()

,artist_id,specialty_id
0,0,0
1,0,1
2,0,2
3,0,3
4,1,2


In [59]:
Special2 = Special.pivot_table(index = "artist_id", columns = "specialty_id", aggfunc = "size", fill_value = 0)
Special2 = Special2.add_prefix("Specialty_").reset_index(level=0)
Special2 = Special2.reset_index(drop=True)
Special2.columns.name = None
Special2

,artist_id,Specialty_0,Specialty_1,Specialty_2,Specialty_3,Specialty_4,Specialty_5,Specialty_6,Specialty_7,Specialty_8,...,Specialty_26,Specialty_27,Specialty_28,Specialty_29,Specialty_30,Specialty_31,Specialty_32,Specialty_33,Specialty_34,Specialty_35
0,0,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,561,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
230,564,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
231,565,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
232,566,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [60]:
Artist2['artist_id'] = Artist2['artist_id'].astype(int) 
Artist2.head()

,artist_id,picture,birthplace,deathplace,birthdate,deathdate
0,0,0.0,0.0,342.0,1853-03-30,1890-07-29
1,1,1.0,1.0,343.0,1841-02-25,1919-12-03
2,2,2.0,2.0,344.0,1840-11-14,1926-12-05
3,3,3.0,3.0,NaN,1932-04-19,NaN
4,4,4.0,4.0,2.0,1824-05-11,1904-01-10


In [61]:
df_list21 = [df1, Artist2, Education2, Places2, Movement2, Special2, Education2]

In [62]:
# Merge the dataframes on the 'artist_id' column using a left join
merged_df = pd.merge(df_list21[0], df_list21[1], on='artist_id', how='left')
for i in range(2, len(df_list21)):
    merged_df = pd.merge(merged_df, df_list21[i], on='artist_id', how='left')
    
merged_df.shape

(13961, 205)

In [63]:
merged_df.head()

,id,name,image_url,artist_id,rating,AI_Generated,logit,time,edge_score,contrast_score,...,Education_38_y,Education_39_y,Education_40_y,Education_41_y,Education_42_y,Education_43_y,Education_44_y,Education_45_y,Education_46_y,Education_47_y
0,0,Cafe Terrace on the Place du Forum,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/Cafe+Terrace+on+the+Place+du+Forum.jpg,0,8.0,0,-8.378802,6.175854,46.383220,55.552072,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,Starry Night,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/Starry+Night.jpg,0,9.0,0,-14.644643,6.019991,47.354136,54.624181,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,A Digger,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/A+Digger.jpg,0,7.0,0,-4.634872,5.785132,28.550276,48.946494,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,A Group of Cottages,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/A+Group+of+Cottages.jpg,0,6.0,0,-10.061778,6.172345,67.087002,52.743048,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,A Pair of Shoes,https://kuleuven-datathon-2023.s3.eu-central-1.amazonaws.com/images/Vincent+Van+Gogh/A+Pair+of+Shoes.jpg,0,8.0,0,-5.156153,6.156727,58.546688,54.431662,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


   # Classifiction

In [64]:
classification = []
for row in merged_df['logit']:
    if row > -5.441349:
        classification.append(1)
    else:
        classification.append(0)
len(classification)

13961

In [65]:
merged_df['classification'] = classification

In [66]:
accuracy_prediction = []
merged_df['accuracy_prediction'] = np.where(merged_df['classification'] == merged_df['AI_Generated'], 1, 0)

In [67]:
np.mean(merged_df['accuracy_prediction'])

0.623451042188955

In [68]:
merged_df.to_csv("new_final_df.csv")

In [70]:
np.sum(merged_df["AI_Generated"] == 1)

3447